In [ ]:
from source_files_extended import *
from plot_helpers import *
from raster_compare.plots import PlotBase

import query_helpers

import math
import pandas as pd

# Analysis

In [ ]:
hillshade_snow_on, hillshade_snow_free = load_hillshade()

df = pd.DataFrame({
        'sfm_snow_depth': sfm_snow_depth_values.ravel(),
        'hillshade_snow_on': hillshade_snow_on.ravel(),
        'hillshade_snow_free': hillshade_snow_free.ravel(),
        'casi_class': casi_classification.ravel(),
})

In [ ]:
df.dropna(inplace=True, how='any', subset=['sfm_snow_depth', 'hillshade_snow_on', 'hillshade_snow_free' ])
df['hillshade_snow_on'] = df['hillshade_snow_on'].astype('int16')
df['hillshade_snow_free'] = df['hillshade_snow_free'].astype('int16')
df['casi_class'] = pd.cut(df['casi_class'], CASI_MAPPING, labels=CASI_CLASSES)

positive_sfm = query_helpers.get_positive(df, 'sfm_snow_depth')
negative_sfm = query_helpers.get_negative(df, 'sfm_snow_depth')

vegetation = ((negative_sfm.casi_class == 'Vegetation') | (negative_sfm.casi_class == 'Water'))
open_areas = ~vegetation

## Hillshade comparison (No Vegetation)

## Snow free SfM

Azimuth: 247

Altitude: 32

In [ ]:
HILLSHADE_MIN = 0
HILLSHADE_MAX = 255

hillshade_range = np.arange(HILLSHADE_MIN, HILLSHADE_MAX + 1, 1)

min_depth = math.floor(negative_sfm.sfm_snow_depth.min())
bins = np.arange(min_depth, 0.5, 0.5)

hist_opts = dict(
    bins=[hillshade_range, bins],
    vmin=0,
    vmax=150,
    cmin=1,
)

In [ ]:
hillshade_snow_free = negative_sfm[open_areas].groupby('hillshade_snow_free')
hillshade_snow_free_median = hillshade_snow_free.median()
hillshade_snow_free_count = hillshade_snow_free.count()

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(15,24))

ax1.set_facecolor('whitesmoke')
ax1.set_facecolor('whitesmoke')
ha = ax1.hist2d(
    negative_sfm[open_areas].hillshade_snow_free,
    negative_sfm[open_areas].sfm_snow_depth,
    **hist_opts,
)
PlotBase.insert_colorbar(ax1, ha[3], 'count')
ax1.set_ylabel(SNOW_DEPTH_LABEL)
ax1.set_title('Negative values with Snow Free hillshade')

ax2.scatter(hillshade_snow_free_median.index, hillshade_snow_free_median.sfm_snow_depth)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference')

ax3.scatter(hillshade_snow_free_count.index, hillshade_snow_free_count.sfm_snow_depth)
ax3.set_ylabel('Count')
ax3.set_title('Count by hillshade')

ax3.set_xlim(HILLSHADE_MIN, HILLSHADE_MAX)
ax3.set_xlabel('Hillshade');

## SfM snow on

Azimuth: 100

Altitude: 47

In [ ]:
hillshade_snow_on = negative_sfm[open_areas].groupby('hillshade_snow_on')
hillshade_snow_on_median = hillshade_snow_on.median()
hillshade_snow_on_count = hillshade_snow_on.count()

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(15,24))

ax1.set_facecolor('whitesmoke')
ax1.set_facecolor('whitesmoke')
ha = ax1.hist2d(
    negative_sfm[open_areas].hillshade_snow_on,
    negative_sfm[open_areas].sfm_snow_depth,
    **hist_opts,
)
PlotBase.insert_colorbar(ax1, ha[3], 'count')
ax1.set_ylabel(SNOW_DEPTH_LABEL)
ax1.set_title('Negative values with Snow On hillshade')

ax2.scatter(hillshade_snow_on_median.index, hillshade_snow_on_median.sfm_snow_depth)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference')

ax3.scatter(hillshade_snow_on_count.index, hillshade_snow_on_count.sfm_snow_depth)
ax3.set_ylabel('Count')
ax3.set_title('Count by hillshade')

ax3.set_xlim(HILLSHADE_MIN, HILLSHADE_MAX)
ax3.set_xlabel('Hillshade');